In [ ]:
import sys; sys.path.append('..')
import mesh, loads, meshing
import numpy as np
import tri_mesh_viewer
import elastic_solid, energy
import loads, sim_utils

In [ ]:
m3d = mesh.Mesh(*meshing.tetrahedralize_extruded_polylines([np.array([(0, 0), (0, 1), (1, 1), (1, 0), (0, 0)])], [], thickness=10, maxVol=0.01), degree=2)

es = elastic_solid.ElasticSolid(m3d, energy.IsotropicLinearElastic(3, 1, 0.3))
traction = loads.Traction(es)

# Apply compression to the four bar sides
bdryC = es.mesh().boundaryMesh().barycenters()
t = traction.boundaryTractions.copy()
t[[c in sim_utils.getBBoxFaceRegion(es, sim_utils.BBoxFace.MIN_X) for c in bdryC]] = [ 1.0, 0, 0]
t[[c in sim_utils.getBBoxFaceRegion(es, sim_utils.BBoxFace.MAX_X) for c in bdryC]] = [-1.0, 0, 0]
t[[c in sim_utils.getBBoxFaceRegion(es, sim_utils.BBoxFace.MIN_Y) for c in bdryC]] = [ 0, 1.0, 0]
t[[c in sim_utils.getBBoxFaceRegion(es, sim_utils.BBoxFace.MAX_Y) for c in bdryC]] = [ 0,-1.0, 0]
traction.boundaryTractions = 0.5 * t

# bm = es.mesh().boundaryMesh()
# writer = mesh.MSHFieldWriter('traction.msh', bm.vertices(), bm.elements())
# writer.addField('t', t)
# del writer

In [ ]:
# Just pin rigid motion
fixedVars = [15, 16, 17, # xyz of vertex at (0, 0, -5)  (pin translation)
             3,  4,     # xy  of vertex at (0, 0, 5)   (pin rotation around y and x axes)
             12        ] # x   of vertex at (0, 1, -5)  (pin rotation around z axis)

In [ ]:
# Block the expansion/contraction of the two ends
fixedVars = [var for var in sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MIN_Z) +
                            sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MAX_Z) if var % 3 == 2]
fixedVars.extend([15, 16, 12]) # also fix xy coordinate of one vertex and pin rotation around z axis

In [ ]:
es.computeEquilibrium([traction], fixedVars=fixedVars)

In [ ]:
v = tri_mesh_viewer.TriMeshViewer(es, wireframe=True)
v.show()

In [ ]:
e_zz = [es.greenStrain(ei)[2, 2] for ei in range(es.numElements())]

In [ ]:
v.update(scalarField=e_zz)

In [ ]:
from matplotlib import pyplot as plt
plt.hist(e_zz, bins=100);